In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
hf_token = ""

In [3]:
from huggingface_hub import login

login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = (
    True  # Use 4bit quantization to reduce memory usage. Can be False.
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

meta-llama/Llama-3.1-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.11.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
{}

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    roles = examples["role"]
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for role, instruction, input, output in zip(
        roles, instructions, inputs, outputs
    ):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = (
            alpaca_prompt.format(role, instruction, input, output) + EOS_TOKEN
        )
        texts.append(text)
    return {
        "text": texts,
    }


# Loading the dataset
from datasets import load_dataset

dataset = load_dataset("ThanhTranVisma/blueVi-GPT-dataset", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

blueVi-GPT-dataset.json:   0%|          | 0.00/173k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/664 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [10]:
train_output = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 32 | Num Epochs = 25
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.241000
2,1.220500
3,1.214200
4,1.122200
5,1.098600
6,0.992300
7,1.025100
8,0.909000
9,0.884900
10,0.805300


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-673234f7-46f2264d4251ca577de86ead;5a1737e5-1803-4e1e-90a6-bf3fddc5d44b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.1-8B-Instruct.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.1-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


<a name="Inference"></a>
### Inference


In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
{}

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Preparing the inputs with the role, instruction, and input
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "What is your name?",  # instruction
            "Can you tell me your name?",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

# Stream the output
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
What is your name?

### Input:
Can you tell me your name?

### Response:
I am bluevi-GPT.<|eot_id|>


In [20]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Preparing the inputs with the role, instruction, and input
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "user",  # role
            "What is your name?",  # instruction
            "Can you tell me your name?",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

# Stream the output
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
user

### Instruction:
What is your name?

### Input:
Can you tell me your name?

### Response:
I'm bluevisprong.<|eot_id|>


In [12]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Preparing the inputs with the role, instruction, and input
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "user",  # role
            "Return the most suitable instruction",  # instruction
            "Please provide the details to create a new operation.",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
user

### Instruction:
Return the most suitable instruction

### Input:
Please provide the details to create a new operation.

### Response:
handle_operation<|eot_id|>


In [21]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Preparing the inputs with the role, instruction, and input
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "Return the most suitable instruction",  # instruction
            "Please provide the details to create a new operation.",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
Return the most suitable instruction

### Input:
Please provide the details to create a new operation.

### Response:
Handle Operating<|eot_id|>


In [13]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "Anonymize the data",  # instruction
            "llice Smith's email is allice.smith@example..com. Her BSN is 12345678. Her address is 15 Elm Street, and her zip code is 12345.",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
Anonymize the data

### Input:
llice Smith's email is allice.smith@example..com. Her BSN is 12345678. Her address is 15 Elm Street, and her zip code is 12345.

### Response:
[NAME_1]'s email is [EMAIL_1]. Her BSN is [BSN_1]. Her address is [ADDRESS_1], and her zip code is [ZIP_1].<|eot_id|>


In [14]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "Detect personal data then return True or False",  # instruction
            "llice Smith's email is allice.smith@example..com. Her BSN is 12345678. Her address is 15 Elm Street, and her zip code is 12345.",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
Detect personal data then return True or False

### Input:
llice Smith's email is allice.smith@example..com. Her BSN is 12345678. Her address is 15 Elm Street, and her zip code is 12345.

### Response:
True<|eot_id|>


In [15]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "Detect personal data then return True or False",  # instruction
            "Hi, how are you today. I love internet and cats",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
Detect personal data then return True or False

### Input:
Hi, how are you today. I love internet and cats

### Response:
False<|eot_id|>


In [16]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "user",  # role
            "Handle Operating",  # instruction
            "Create a new operation called 'Webinar' with no invoicing applied, unit price 50",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
user

### Instruction:
Handle Operating

### Input:
Create a new operation called 'Webinar' with no invoicing applied, unit price 50

### Response:
In order to create the 'Webinar' operation, please provide the following details:
1. Duration (in minutes)
2. Methods of Consultation (TEL, WEB, LOC)<|eot_id|>


In [17]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "assistant",  # role
            "Handle Operating",  # instruction
            "Create a new operation called 'Webinar' with no invoicing applied, unit price 50",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Role:
assistant

### Instruction:
Handle Operating

### Input:
Create a new operation called 'Webinar' with no invoicing applied, unit price 50

### Response:
{
  "name": "Webinar",
  "invoice_description": "",
  "duration": null,
  "wizard": null,
  "invoicing": false,
  "hourlyRate": null,
  "unitPrice": 50,
  "operationRateType": 0,
  "methodsOfConsult": null
}<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [18]:
model.save_pretrained("Llama3.1-8B-blueVi-GPT-lora-v03")
tokenizer.save_pretrained("Llama3.1-8B-blueVi-GPT-lora-v03")
# Push both model and tokenizer to the same repo on Hugging Face Hub
model.push_to_hub(
    "ThanhTranVisma/Llama3.1-8B-blueVi-GPT-lora-v03", token=hf_token
)
tokenizer.push_to_hub(
    "ThanhTranVisma/Llama3.1-8B-blueVi-GPT-lora-v03", token=hf_token
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67323506-014c45364191f11c1a14e199;410b7bae-38ea-4447-ad98-e8559c32c4a5)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.1-8B-Instruct.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.1-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67323508-629ec85447cffae76a4f813b;0b15db98-f457-4ad3-934a-296bc629d29c)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.1-8B-Instruct.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/ThanhTranVisma/Llama3.1-8B-blueVi-GPT-lora-v03


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### GGUF / llama.cpp Conversion

In [19]:
model.push_to_hub_gguf(
    "ThanhTranVisma/Llama3.1-8B-blueVi-GPT-v03",
    tokenizer,
    quantization_method=["q4_k_m", "q8_0", "q5_k_m"],
    token=hf_token,
)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.95 out of 50.99 RAM for saving.


100%|██████████| 32/32 [00:27<00:00,  1.16it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/ThanhTranVisma/Llama3.1-8B-blueVi-GPT-v03
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/ThanhTranVisma/Llama3.1-8B-blueVi-GPT-v03
